In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import ee
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/home/google_cred.json')

In [3]:
# GEE authentication using service account key stored in local directory
ee.Initialize(credentials)

In [4]:
import geemap
from ipyleaflet import CircleMarker, Polygon, LayersControl, DrawControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout, Accordion
import ipywidgets as widgets
import sys, asyncio, codecs, json, time
import numpy as np

In [5]:
# STATUS

def start_processing(display_text):
    status_display.set_trait('value', '<center style="color:#f00000; font-size:14pt; font-weight:bold">{}</center>'.format(display_text))
def end_processing():
    update_narrativedisplay()
    status_display.set_trait('value', '<center style="color:#00d000; font-size:14pt; font-weight:bold">READY</center>')

In [6]:
# Create decorator to prevent events triggers while slider is being moved
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [7]:
# HAZARD INFO

hazards = {
    'ds': {
        'keyname': 'ds',
        'short_displayname': 'DS',
        'displayname': 'dryspells',
        'unit': 'events',
        'min': 0,
        'max': 100,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 100,
        'bin_width': 1,
        'hazard_category': 'drought',
        'narrative_phrase': 'as many as {:.0f} dryspells'
    },
    'maxdryspell': {
        'keyname': 'maxdryspell',
        'short_displayname': 'mxDS',
        'displayname': 'max dryspell duration',
        'unit': 'days',
        'min': 0,
        'max': 365,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 73,
        'bin_width': 5,
        'hazard_category': 'drought',
        'narrative_phrase': 'dryspells of up to {:.0f} days'
    },
    'mtt35': {
        'keyname': 'mtt35',
        'short_displayname': 'MTT35',
        'displayname': 'days temp above 35',
        'unit': 'days',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 74,
        'bin_width': 5,
        'hazard_category': 'heat',
        'narrative_phrase': "as many as {:.0f} days above 35 C"
    },
    'maxtemp': {
         'keyname': 'maxtemp',
        'short_displayname': 'mxT',
         'displayname': 'maximum annual temperature',
        'unit': 'C',
         'min': -10,
         'max': 60,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 70,
         'bin_width': 1,
         'hazard_category': 'heat',
        'narrative_phrase': 'temperatures as high as {:.0f} C'
    },
    'mintemp': {
         'keyname': 'mintemp',
        'short_displayname': 'mnT',
         'displayname': 'minimum annual temperature',
        'unit': 'C',
         'min': -110,
         'max': 46,
         'use_greaterthan': False,
         'years': list(range(2020, 2100)),
         'num_bins': 78,
         'bin_width': 2,
         'hazard_category': 'cold',
        'narrative_phrase': 'temperatures as low as {:.0f} C'
    },
    'modlandslide': {
        'keyname': 'modlandslide',
        'short_displayname': 'LS',
        'displayname': 'days moderate landslide risk',
        'unit': 'days',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 60,
        'bin_width': 5,
        'hazard_category': 'landslide',
        'narrative_phrase': 'as many as {:.0f} days of moderate-to-high landslide risk'
    },
    'totalprecip': {
         'keyname': 'totalprecip',
         'short_displayname': 'totPR',
         'displayname': 'annual total precipitation',
        'unit': 'mm',
         'min': 0,
         'max': 8500,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 85,
         'bin_width': 100,
         'hazard_category': 'precipitation',
        'narrative_phrase': 'as much as {:.0f} mm total precipitation in a year'
    },
    'maxprecip': {
        'keyname': 'maxprecip',
        'short_displayname': 'mxPR',
         'displayname': 'maximum daily precipitation',
        'unit': 'mm',
         'min': 0,
         'max': 4000,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 80,
         'bin_width': 20,
         'hazard_category': 'precipitation',
        'narrative_phrase': 'as many as {:.0f} mm precipitation in a day'
    },
    'riverflood': {
         'keyname': 'riverflood',
        'short_displayname': 'riverFLD',
         'displayname': 'riverine inundation',
        'unit': 'meters',
         'min': 0,
         'max': 5,
         'use_greaterthan': True,
         'years': [1980, 2030, 2050, 2080],
         'num_bins': 10,
         'bin_width': 0.2,
         'hazard_category': 'flooding',
        'narrative_phrase': 'as much as {:.1f} m of inundation'
    },
    'coastalflood': {
         'keyname': 'coastalflood',
        'short_displayname': 'coastalFLD',
         'displayname': 'coastal inundation',
        'unit': 'meters',
         'min': 0,
         'max': 5,
         'use_greaterthan': True,
         'years': [1980, 2030, 2050, 2080],
         'num_bins': 10,
         'bin_width': 0.2,
         'hazard_category': 'flooding',
        'narrative_phrase': 'as much as {:.1f} m of inundation'
    }
}

shortname_to_hazname = {
    hazards[hn]['short_displayname']: hn for hn in hazards.keys()
}

FLOOD_YEARS = [2030, 2050, 2080]

HAZARD_CATEGORIES = ['heat', 'drought', 'landslide', 'precipitation', 'flooding', 'cold']

hazard_data = {hazard: {
    'probs': ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/{}'.format(hazard)),
    'tprobs': ee.ImageCollection('users/tedwongwri/dataportal/thresholdposterior_probs/{}'.format(hazard)),
    }
    for hazard in hazards.keys()
}

HAZARD_THRESHOLD_PERCENTILE = [10, 90]

LOW_MODERATE_CUTOFF = 40
MODERATE_HIGH_CUTOFF = 60
HIGH_VERYHIGH_CUTOFF = 80

all_percentile_data = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/hazards')
tmp = ee.List([])
for hazard_name in hazards.keys():
    if hazards[hazard_name]['hazard_category'] in HAZARD_CATEGORIES:
        tmp = tmp.add(all_percentile_data.filterMetadata('hazard', 'equals', hazard_name).select('{0}_p{1}'.format(hazards[hazard_name]['keyname'], str([HAZARD_THRESHOLD_PERCENTILE[int(hazards[hazard_name]['use_greaterthan'])], 95][int(hazards[hazard_name]['hazard_category']=='flooding')]))).first())
percentile_data = ee.ImageCollection(tmp).toBands().rename([i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES])

In [8]:
# DATA AND INITIAL SETTINGS

initial_threshold = 30
min_threshold = 5
max_threshold = 50
threshold_interval = 5
DEFAULT_CURRENTYEAR = 2022
currentYear = DEFAULT_CURRENTYEAR
futureYear = 2050
prev_threshold_current = initial_threshold
prev_threshold_future = initial_threshold
prev_futureYear = 2050

prev_layers = []

persistent_threshold_value = initial_threshold
persistent_ev_current = None
persistent_ev_future = None
persistent_threshold_value = initial_threshold
#initial_coords = [50.85045, 4.34878]  # Brussels
initial_coords = [-70.575, -33.383333] # Vitacura  -- Give as lat, lon
current_geom = ee.Geometry.Point(initial_coords)
current_location_marker = None
ev_val = 'expected'
mode_is_ev = True

auto_thresholds = True
percentilebased_lookup_thresholds = {
    hn: hazards[hn]['min'] for hn in hazards.keys()
}
hazard_threshold_sliders = {}
userdefined_lookup_thresholds = {
    hn: hazards[hn]['min'] for hn in hazards.keys()
}
highrisk_categories = {'current': [], 'future': []}
user_ever_defined = False

current_hazard = 'maxdryspell'

MAPprobs = hazard_data[current_hazard]['tprobs']
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

def update_data():
    global MAPprobs
    global currentprobs
    global futureprobs
    MAPprobs = hazard_data[current_hazard]['tprobs']
    currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [9]:
# MAP

def latlon_to_lonlat(coords):
# Converts EE coords (x, y) to ipyleaflet coords (y, x)
# Works for geojson coords
    if type(coords[0]) == float:
        return [coords[1], coords[0]]
    else:
        return [[[i[1],i[0]] for i in j] for j in coords]

upload_button = widgets.FileUpload(
    accept = '.geojson',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple = False,  # True to accept multiple files upload else False
    description = 'GeoJSON'
)

Map = geemap.Map(center=latlon_to_lonlat(initial_coords), zoom=3)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='bottomright'))
draw_control = DrawControl(circle={}, polyline={}, rectangle={}, position='topleft')
Map.add_control(draw_control)

current_location_marker = CircleMarker(location=latlon_to_lonlat(initial_coords), name="location of interest")
Map.add_layer(current_location_marker)

vizParamsC = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', '909000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', 'ff0000'],
  'opacity': 0.5
}
    
def update_location(geom):
    start_processing("Processing location")
    global current_geom
    global current_location_marker
    Map.remove_layer(current_location_marker)
    draw_control.clear()
    new_geom = geom
    if new_geom['type'] == 'Point':
        current_geom = ee.Geometry.Point(new_geom['coordinates'])
        ipl_loc = latlon_to_lonlat(new_geom['coordinates'])
        current_location_marker = CircleMarker(location=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    else:
        current_geom = ee.Geometry.Polygon(new_geom['coordinates'])
        current_location_marker = Polygon(locations=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    update_percentiles()
    update_table()
    update_probplot()
    update_narrativedisplay()
    end_processing()
        
def drawcontrol_update(e):
    update_location(draw_control.last_draw['geometry'])

@debounce(0.5)
def uploadbutton_update(e):
    start_processing("Processing location upload")
    f = upload_button.data[-1]
    file_content = json.loads(codecs.decode(f, encoding="utf-8"))
    update_location(file_content['features'][0]['geometry'])
    polygon_coords = current_geom.coordinates().getInfo()[0]
    exes = np.array([i[0] for i in polygon_coords])
    whys = np.array([i[1] for i in polygon_coords])
    Map.setCenter(lat=np.mean(whys), lon=np.mean(exes))
    Map.set_trait('zoom', 9)
    update_percentiles()
    update_probplot()
    update_table()
    update_narrativedisplay()
    end_processing()
'''    
@debounce(0.5)
def marker_update(e):
    debug_display.set_trait('value', str(e))
    update_probplot()
    update_table()
    update_percentiles()
'''

draw_control.observe(drawcontrol_update)
upload_button.observe(uploadbutton_update)

def update_map():
# Updates map w new prob layers according to futureYear and whichever thresholds are appropriate given mode_is_ev state
# Updates some globals
    global prev_futureYear
    global prev_threshold_current
    global prev_threshold_future
    global prev_layers
    
    #for layername in prev_layers:
    #   Map.remove_ee_layer(name=layername)
        
    if mode_is_ev:
        c_ev, dummy = ev_prob(currentYear)
        f_ev, dummy = ev_prob(futureYear)
        c_threshold = hazards[current_hazard]['min'] + (round(c_ev / hazards[current_hazard]['bin_width']) * hazards[current_hazard]['bin_width'])
        f_threshold = hazards[current_hazard]['min'] + (round(f_ev / hazards[current_hazard]['bin_width']) * hazards[current_hazard]['bin_width'])
    else:
        c_threshold = threshold_slider.value
        f_threshold = threshold_slider.value
        
    new_visparamsC = {
      'bands': ['threshold_{}'.format(str(c_threshold).replace('-', 'neg'))],
      'min': 0,
      'max': 1,
      'palette': ['000000', '909000'],
      'opacity': 0.5
    }
    new_visparamsF = {
      'bands': ['threshold_{}'.format(str(f_threshold).replace('-', 'neg'))],
      'min': 0,
      'max': 1,
      'palette': ['000000', 'ff0000'],
      'opacity': 0.5
    }
    currentlayername = str(currentYear) + ' Prob({0} {1} {2})'.format(hazards[current_hazard]['displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(c_threshold))
    Map.add_ee_layer(currentprobs, new_visparamsC, currentlayername)
    futurelayername = str(futureYear) + ' Prob({0} {1} {2})'.format(hazards[current_hazard]['displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(f_threshold))
    Map.add_ee_layer(futureprobs, new_visparamsF, futurelayername)
    prev_layers = [currentlayername, futurelayername]

In [10]:
# HAZARD DISPLAY

def nearest_datayear(year, hazard_name):
    if year in hazards[hazard_name]['years']:
        return year
    diffs = [abs(year-i) for i in FLOOD_YEARS]
    min_diff = min(diffs)
    return FLOOD_YEARS[diffs.index(min_diff)]
    

def update_hazard(e):
    start_processing("Processing hazard change")
    global current_hazard
    global currentYear
    global futureYear
    current_hazard = hazard_selector.value
    if hazards[current_hazard]['hazard_category'] == 'flooding':
        currentYear = nearest_datayear(currentYear, 'riverflood')
        futureYear = nearest_datayear(futureYear, 'riverflood')
        futureyear_slider.value = (currentYear, futureYear)
    else:
        currentYear = DEFAULT_CURRENTYEAR
    update_data()
    update_table()
    #update_map()
    update_probplot()
    update_narrativedisplay()
    update_thresholdslider()
    end_processing()
    
hazard_selector = widgets.Dropdown(
    options=[('{0} ({1})'.format(hazards[h]['displayname'], hazards[h]['short_displayname']), h) for h in hazards.keys()],
    value=current_hazard,
    description='',
    disabled=False,
)
hazard_display = widgets.HTML()
indicator_display = widgets.HTML()

hazard_selector.observe(update_hazard)

In [11]:
# SELECTORS

@debounce(0.5)
def threshold_update(e):
    global persistent_threshold_value
    persistent_threshold_value = threshold_slider.value
    threshold_slider.description = '{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] )
    update_table()
    #update_map()
    
@debounce(0.5)
def futureyear_update(e):
    start_processing("Processing change of year")
    global futureYear
    global currentYear
    global futureprobs
    global prev_futureYear
    global persistent_ev_future
    if hazards[current_hazard]['hazard_category'] == 'flooding':
        futureyear_slider.value = (nearest_datayear(futureyear_slider.value[0], 'riverflood'), nearest_datayear(futureyear_slider.value[1], 'riverflood'))
    currentYear = futureyear_slider.value[0]
    futureYear = futureyear_slider.value[1]
    update_data()
    #persistent_ev_future = get_ev(futureYear)
    
    update_probplot()
    update_table()
    #update_map()
    update_percentiles()
    update_narrativedisplay()
    prev_futureYear = futureYear
    end_processing()
      
@debounce(0.5)
def mode_update(e):
    start_processing("Processing change of threshold selection mode")
    global mode_is_ev
    global persistent_ev_current
    global persistent_ev_future
    global plot_box
    mode_toggle.tooltips = ['What {} can you expect?'.format(hazards[current_hazard]['short_displayname']), 'How probable is a specific {} level?'.format(hazards[current_hazard]['short_displayname'])]
    mode_is_ev = mode_toggle.value == ev_val
    if mode_is_ev:
    #    if not persistent_ev_current:
    #        persistent_ev_current = get_ev(currentYear)
    #    if not persistent_ev_future:
    #        persistent_ev_future = get_ev(futureYear)
        threshold_slider.layout.visibility = 'hidden'
        probplot.canvas.layout.visibility = 'hidden'
    else:
        threshold_slider.layout.visibility = 'visible'
        probplot.canvas.layout.visibility = 'visible'
        probplot.set_size_inches(5,4)
        
    update_table()
    #update_map()
    end_processing()

futureyear_slider = widgets.SelectionRangeSlider(
    value = (currentYear, futureYear),
    options = [(str(i), i) for i in range(2020, 2081)],
    description = 'Years',
    continuous_update = False,
    disabled=False
)

threshold_intslider = widgets.IntSlider(
    value=initial_threshold,
    min=0,
    max=50,
    step=5,
    description='{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] ),
    continuous_update=False,
    disabled=False
)
threshold_floatslider = widgets.FloatSlider(
    value = 0.4,
    min = 0.0,
    max = 2.0,
    step = 0.2,
    description = '{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] ),
    continuous_update=False,
    disabled=False
)

threshold_intslider.layout.visibility = ['visible', 'hidden'][mode_is_ev * 1]
threshold_floatslider.layout.visibility = 'hidden'

threshold_slider = threshold_intslider

def update_thresholdslider():
    global threshold_slider
    if hazards[current_hazard]['bin_width'] < 1:
        threshold_slider = threshold_floatslider
        threshold_floatslider.layout.visibility = ['visible', 'hidden'][mode_is_ev * 1]
        threshold_intslider.layout.visibility = 'hidden'
    else:
        threshold_slider = threshold_intslider
        threshold_intslider.layout.visibility = ['visible', 'hidden'][mode_is_ev * 1]
        threshold_floatslider.layout.visibility = 'hidden'
    threshold_slider.max = hazards[current_hazard]['bin_width'] * hazards[current_hazard]['num_bins']
    threshold_slider.min = hazards[current_hazard]['min']
    threshold_slider.step = hazards[current_hazard]['bin_width']
    threshold_slider.description='{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] )

mode_toggle = widgets.ToggleButtons(
    options=[ev_val, 'set magnitude'],
    description='',
    #disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['What {} can you expect?'.format(hazards[current_hazard]['short_displayname']), 'How probable is a specific {} level?'.format(hazards[current_hazard]['short_displayname'])],
    value = ['set magnitude', ev_val][int(mode_is_ev)],
    disabled=False
)

table_display = widgets.HTML()

threshold_intslider.observe(threshold_update)
threshold_floatslider.observe(threshold_update)
futureyear_slider.observe(futureyear_update)
mode_toggle.observe(mode_update)

In [12]:
# NARRATIVE DISPLAY

narrative_display = widgets.HTML()

def pluralizer(ll, mode):
    if len(ll) > 1:
        return ["are", "s"][int(mode == "ess")]
    else:
        return ["is", ""][int(mode == "ess")]
    
def update_narrativedisplay():
    html = "<h3>Hazards</h3>"
    if len(highrisk_categories['current']) > 0:
        html += "The most urgent hazard{0} for this city {1} <b>currently</b><ol><li>{2}</li></ol>".format(pluralizer(highrisk_categories['current'], 'ess'), pluralizer(highrisk_categories['current'], "isare"), "</li><li>".join(highrisk_categories['current']))
    else:
        html += "Currently there is low risk for all hazard categories."
    if len(highrisk_categories['future']) > 0:
        html += "In <b>{0}</b>, the most urgent hazard{1} will be <ol><li>{2}</li></ol>".format(str(futureYear), pluralizer(highrisk_categories['future'], "ess"), "</li><li>".join(highrisk_categories['future']))
    else:
        html += "There is low risk for all hazard categories in {0}.".format(str(futureYear))
    
    
    
    html += "<br>"
    html += "You can <b>currently</b> expect...<ul>"
    for haz in highrisk_categories['currenthaz']:
        expected_haz, dummy = ev_prob(nearest_datayear(currentYear, haz), haz)
        html += "<li>{}</li>".format(hazards[haz]["narrative_phrase"].format(expected_haz * [1, 0.1][hazards[haz]['hazard_category']=='flooding']))
    html += "</ul>"
    html += "In <b>{}</b> you can expect...<ul>".format(str(futureYear))
    for haz in highrisk_categories['futurehaz']:
        expected_haz, dummy = ev_prob(nearest_datayear(futureYear, haz), haz)
        html += "<li>{}</li>".format(hazards[haz]["narrative_phrase"].format(expected_haz * [1, 0.1][hazards[haz]['hazard_category']=='flooding']))
    html += "</ul>"
        
    narrative_display.set_trait('value', html)


In [13]:
# TABLES

def renameBands(img):
# MAPprob files have bandnames like "threshold_5"
# Returns threshold vals like "05"
    new_bandnames = [i.split('_')[1].replace('neg', '-') for i in img.bandNames().getInfo()]
    max_length = max(len(i) for i in new_bandnames)
    return img.rename(new_bandnames)

def CRFtext(probstring, prob):
    # Assigns label to numerical probability according to Box 4 (p. 44) of
    # https://www.globalcovenantofmayors.org/wp-content/uploads/2019/08/Data-TWG_Reporting-Framework_GUIDENCE-NOTE_FINAL.pdf
    if prob > 0.05:
        return 'high'
    elif prob > 0.005:
        return 'moderate'
    elif prob > 0.0005:
        return 'low'
    else:
        return 'do not know'
    
def changetext(before, after, inv):
    if before == after:
        return 'no change'
    chg = after > before
    if inv:
        chg = not chg
    return ['decrease', 'increase'][chg * 1]

def ev_prob(year, hazard=current_hazard):
# Plot expected value of hazard for given year
    probs = hazard_data[hazard]['probs'].filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
    if np.sum([float(i) for i in probs.values()]) == 0:
        return 0, 1
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0].replace('neg', '-')) + float(i[1].replace('neg', '-'))) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    for idx in range(len(mag_range)):
        mag_pair = mag_range[idx]
        if ev >= float(mag_pair[0].replace('neg', '-')) and ev < float(mag_pair[1].replace('neg', '-')):
            break
    key = list(probs.keys())[idx]
    prob = probs[key]
    return ev, prob
    
def update_table():
# Inserts output HTML into table_display widget
# Different HTML depending on whether mode_is_ev
    if mode_is_ev:
        current_ev, current_prob = ev_prob(nearest_datayear(currentYear, current_hazard))
        future_ev, future_prob = ev_prob(nearest_datayear(futureYear, current_hazard))
        evstring_current = '{0:.1f} {1}'.format(current_ev, hazards[current_hazard]['unit'])
        evstring_future = '{0:.1f} {1}'.format(future_ev, hazards[current_hazard]['unit'])
       
    else:
        current_probs = hazard_data[current_hazard]['tprobs'].filterMetadata('year', 'equals', nearest_datayear(currentYear, current_hazard)).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
        future_probs = hazard_data[current_hazard]['tprobs'].filterMetadata('year', 'equals', nearest_datayear(futureYear, current_hazard)).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
        for k in current_probs.keys():
            if current_probs[k] is None:
                current_probs[k] = 0
        for k in future_probs.keys():
            if future_probs[k] is None:
                future_probs[k] = 0
        thresholds = [float(i.split('_')[1].replace('neg', '-')) for i in current_probs.keys()]
        
        for idx in range(len(thresholds)):
            mag_pair = (thresholds[idx], thresholds[idx] + hazards[current_hazard]['bin_width'])
            if persistent_threshold_value >= mag_pair[0] and persistent_threshold_value < mag_pair[1]:
                break
        key = list(current_probs.keys())[idx]    
        current_prob = current_probs[key]
        future_prob = future_probs[key]
    
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_current += ' (' + CRFtext(probstring_current, current_prob) + ')'
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    probstring_future += ' (' + CRFtext(probstring_future, future_prob) + ')'
    if current_prob > 0:
        ri_string_current = '{:.1f} years'.format(1.0/current_prob)  # recurrence interval
    else:
        ri_string_current = '(unprecedented event)'
    if future_prob > 0:
        ri_string_future = '{:.1f} years'.format(1.0/future_prob)
    else:
        ri_string_future = '(unprecedented event)'
    if mode_is_ev:
        newHtml = '<br /><b>What ' + hazards[current_hazard]['short_displayname'] + ' can you expect?</b><table style="width:600px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>magnitude</td><td>magnitude change</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(nearest_datayear(currentYear, current_hazard)) + '</td><td>' + evstring_current + '</td><td rowspan="2">' + changetext(current_ev, future_ev, False) + '</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(nearest_datayear(futureYear, current_hazard)) + '</td><td>' + evstring_future + '</td></tr></table>'
    else:
        newHtml = '<br /><b>How probable is {0} {1} {2}?</b><table style="width:500px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>probability</td><td>recurrence interval</td><td>probability change</td></tr>'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(persistent_threshold_value))
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(nearest_datayear(currentYear, current_hazard)) + '</td><td style="text-align: center">' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(nearest_datayear(futureYear, current_hazard)) + '</td><td style="text-align: center">' + probstring_future + '</td><td>' + ri_string_future + '</td></tr></table>'
    
    table_display.set_trait('value', newHtml)

In [14]:
# PLOT

def update_probplot():
# Replots plots of probability vs EHE threshold value
    probplot.canvas.draw()
    probplot.canvas.flush_events()
    global current_plot
    global future_plot
    if current_plot:
        dummy = current_plot.pop(0)
        dummy.remove()
    if future_plot:
        dummy = future_plot.pop(0)
        dummy.remove()
    current_data = renameBands(currentprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    for k in current_data.keys():
        if current_data[k] is None:
            current_data[k] = 0
    xvals = list(current_data.keys())
    xvals.sort(key=lambda x: float(x))
    current_yvals = [current_data[i] for i in xvals]
    current_plot = plt.plot([float(i) for i in xvals], [100 * i for i in current_yvals], label=str(currentYear), color='green')
    future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    for k in future_data.keys():
        if future_data[k] is None:
            future_data[k] = 0
    future_yvals = [future_data[i] for i in xvals]
    future_plot = plt.plot([float(i) for i in xvals], [100 * i for i in future_yvals], label=str(futureYear), color='lime')
    plt.xlabel('{0} ({1}) {2} threshold'.format(hazards[current_hazard]['short_displayname'], hazards[current_hazard]['unit'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1]))
    plt.legend()
    keys = [float(i) for i in xvals]
    ticks = [keys[i] for i in range(len(keys)) if i % (len(keys) // 10) == 0]
    plt.xticks(ticks)
    plt.xlim((ticks[0], ticks[-1]))
    #probplot.set_size_inches(5,4)
    
plt.ioff()

probplot = plt.figure(figsize=(5,4))
current_plot = None
future_plot = None
plt.xlabel('{0} ({1}) {2} threshold'.format(hazards[current_hazard]['short_displayname'], hazards[current_hazard]['unit'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1]))
plt.ylabel('Probability (%)')

plt.ion();

In [15]:
# PERCENTILES
# Looks up historical 75th percentile
# Finds prob that (max of current/future prob within AOI) >= p95
# Does this for each hazard and color codes display based on max-prob hazard within each hazard class

hazardthreshold_toggle = widgets.ToggleButtons(
    options=['{}th percentiles'.format(HAZARD_THRESHOLD_PERCENTILE[1]), 'custom'],
    description='',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Base hazard risk levels on local {}th percentiles'.format(HAZARD_THRESHOLD_PERCENTILE[1]), 'Manually enter hazard-indicator thresholds'],
    layout=Layout(width='300px')
)

def percentilethreshold_table_html():
    html = "<table><tr><th style='padding-left:3px'>indicator</th><th style='padding-left:20px'>{}th percentile magnitude</th></tr>".format(HAZARD_THRESHOLD_PERCENTILE[1])
    for hn in hazards.keys():
        html += "<tr style='border:1px gray solid'><td>{0}</td><td style='padding-left:20px'>{1} {2}</td></tr>".format(hazards[hn]['displayname'], str(percentilebased_lookup_thresholds[hn] ), hazards[hn]['unit'])
    html += "</table>"
    return html

thresholdvalue_container = widgets.HBox([])
def hazardthreshold_update(e):
    start_processing("Processing change of")
    global user_ever_defined
    global auto_thresholds
    auto_thresholds = hazardthreshold_toggle.value == hazardthreshold_toggle.options[0]
    if not auto_thresholds:
        user_ever_defined = True
    for hn in hazards.keys():
        hazard_threshold_sliders[hn].disabled = auto_thresholds
    if auto_thresholds and not user_ever_defined:
        for hn in hazards.keys():
            hazard_threshold_sliders[hn].value = percentilebased_lookup_thresholds[hn] * [1, 0.1][int(hazards[hn]['hazard_category']=='flooding')]
    else:
        for hn in hazards.keys():
            hazard_threshold_sliders[hn].value = userdefined_lookup_thresholds[hn]
    thresholdvalue_container.children=[[threshold_sliderbox, threshold_percentiletable][auto_thresholds]]
    update_narrativedisplay()
    end_processing()

@debounce(0.5)
def hazardthresholdslider_update(e):
    start_processing("Processing threshold change")
    for hn in hazards.keys():
        userdefined_lookup_thresholds[hn] = hazard_threshold_sliders[hn].value
    update_percentiles()
    end_processing()

hazardthreshold_toggle.observe(hazardthreshold_update)
        
for hn in hazards.keys():
    hazard_threshold_sliders[hn] = [widgets.IntSlider, widgets.FloatSlider][int(hazards[hn]['bin_width'] != round(hazards[hn]['bin_width']))](
        value = hazards[hn]['min'],
        min = hazards[hn]['min'],
        max = hazards[hn]['min'] + hazards[hn]['num_bins'] * hazards[hn]['bin_width'],
        step = hazards[hn]['bin_width'],
        description = '',
        continuous_update = False,
        disabled=True
    )
    hazard_threshold_sliders[hn].observe(hazardthresholdslider_update)
    
threshold_sliderbox = widgets.VBox([widgets.HBox([widgets.Label(value=hazards[hn]['displayname']), hazard_threshold_sliders[hn]]) for hn in hazards.keys()], layout=Layout(width='500px'))
threshold_percentiletable = widgets.HTML(value = percentilethreshold_table_html())
#threshold_sliderbox.layout.visibility = ['visible', 'hidden'][auto_thresholds]

threshold_entry = widgets.HTML()

percentile_codes = {
    i: ('#ffffff', 'low risk', '#000000') if i < LOW_MODERATE_CUTOFF else (('#ffff00', 'moderate risk', '#000000') if i < MODERATE_HIGH_CUTOFF else (('#ff9900', 'high risk', '#ffffff') if i < HIGH_VERYHIGH_CUTOFF else ('#ff0000', 'very high risk', '#ffffff'))) for i in range(101)
}

def update_percentiles():
    global percentilebased_lookup_thresholds
    global highrisk_categories
    global threshold_percentiletable
    local_percentiles = percentile_data.reduceRegion(ee.Reducer.max(), current_geom, scale=250, bestEffort=True).getInfo()
    for k in local_percentiles.keys():
        if local_percentiles[k] is None:
            local_percentiles[k] = 0
    percentilebased_lookup_thresholds = {}
    for hn in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]:
        mags = [hazards[hn]['min'] + (i * hazards[hn]['bin_width']) for i in range(hazards[hn]['num_bins'] + 1)]
        diffs = [local_percentiles[hn] - i for i in mags]
        i = 0
        while i < len(diffs) - 2 and diffs[i] > 0:
            i += 1
        percentilebased_lookup_thresholds[hn] = mags[i]
        
    threshold_percentiletable.set_trait('value', percentilethreshold_table_html())
    if not user_ever_defined:
        for hn in hazards.keys():
            hazard_threshold_sliders[hn].value = percentilebased_lookup_thresholds[hn]# * [1, 0.1][hazards[hn]['hazard_category']=='flooding']
    lookup_source = {
        hn: [userdefined_lookup_thresholds, percentilebased_lookup_thresholds][int(auto_thresholds)][hn] for hn in hazards.keys()
    }
    hazard_threshold_values = {
        hn: round(lookup_source[hn] * [1, 10][hazards[hn]['hazard_category'] == 'flooding']) for hn in hazards.keys()
    }

    local_currentprobs = {
        hazard_name: hazard_data[hazard_name]['tprobs'].filterMetadata('year', 'equals', nearest_datayear(currentYear, hazard_name)).first().reduceRegion(reducer=ee.Reducer.max(), geometry=current_geom, scale=250, bestEffort=True).getInfo() for hazard_name in hazards.keys()
    }
    local_futureprobs = {
        hazard_name: hazard_data[hazard_name]['tprobs'].filterMetadata('year', 'equals', nearest_datayear(futureYear, hazard_name)).first().reduceRegion(reducer=ee.Reducer.max(), geometry=current_geom, scale=250, bestEffort=True).getInfo() for hazard_name in hazards.keys()
    }
    local_current_iszero = {}
    local_future_iszero = {}
    for hazard_name in hazards.keys(): # Deal with situation in which all prob density is at lowest magnitude. Don't want hazard to appear high-risk despite high prob.
        probdictc = hazard_data[hazard_name]['probs'].filterMetadata('year', 'equals', nearest_datayear(currentYear, hazard_name)).first().reduceRegion(reducer=ee.Reducer.max(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
        local_current_iszero[hazard_name] = probdictc[list(probdictc.keys())[0]] == 1
        probdictf = hazard_data[hazard_name]['probs'].filterMetadata('year', 'equals', nearest_datayear(futureYear, hazard_name)).first().reduceRegion(reducer=ee.Reducer.max(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
        local_future_iszero[hazard_name] = probdictf[list(probdictf.keys())[0]] == 1
    current_hazlevel = {}
    current_maxhaz = {}
    for hazcat in HAZARD_CATEGORIES:
        running_maxhaz = None
        running_max = -1
        for hazard_name in hazards.keys():
            if hazards[hazard_name]['hazard_category']==hazcat:
                x = [local_currentprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))], 0][local_currentprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))] is None or local_current_iszero[hazard_name]]
                if x > running_max:
                    running_max = x
                    running_maxhaz = hazard_name
        current_hazlevel[hazcat] = round(100*running_max)
        current_maxhaz[hazcat] = running_maxhaz
    future_hazlevel = {}
    future_maxhaz = {}
    for hazcat in HAZARD_CATEGORIES:
        running_maxhaz = None
        running_max = -1
        for hazard_name in hazards.keys():
            if hazards[hazard_name]['hazard_category']==hazcat:
                x = [local_futureprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))], 0][local_futureprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))] is None or local_future_iszero[hazard_name]]
                if x > running_max:
                    running_max = x
                    running_maxhaz = hazard_name
        future_hazlevel[hazcat] = round(100*running_max)
        future_maxhaz[hazcat] = running_maxhaz
#    current_hazlevel = {
#        hazcat: round(100 * max([[local_currentprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))], 0][local_currentprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))] is None or local_current_iszero[hazard_name]] for hazard_name in hazards.keys() if hazards[hazard_name]['hazard_category']==hazcat])) for hazcat in HAZARD_CATEGORIES
#    }
#    future_hazlevel = {
#        hazcat: round(100 * max([[local_futureprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))], 0][local_futureprobs[hazard_name]['threshold_{}'.format(str(hazard_threshold_values[hazard_name]).replace('-', 'neg'))] is None or local_future_iszero[hazard_name]] for hazard_name in hazards.keys() if hazards[hazard_name]['hazard_category']==hazcat])) for hazcat in HAZARD_CATEGORIES
#    }

    html1 = "<table style='width: 200px'><tr><td><b>{0}</b><td><b>{1}</td></tr>".format(str(currentYear), str(futureYear))
    for hazcat in HAZARD_CATEGORIES:
        html1 += "<tr><td style='padding-left: 2px; border: 1px solid gray; color:{0}; background-color:{1}'>{2}<br /><i style='font-size:9pt'>{3}</i></td>".format(percentile_codes[current_hazlevel[hazcat]][2], percentile_codes[current_hazlevel[hazcat]][0], hazcat, percentile_codes[current_hazlevel[hazcat]][1])
        html1 += "<td style='padding-left: 2px; border: 1px solid gray; color:{0}; background-color:{1}'>{2}<br /><i style='font-size:9pt'>{3}</i></td></tr>".format(percentile_codes[future_hazlevel[hazcat]][2], percentile_codes[future_hazlevel[hazcat]][0], hazcat, percentile_codes[future_hazlevel[hazcat]][1])
    html1 += "</table>"    
    hazard_display.set_trait('value', html1)
    
    html2 = "<table style='width: 200px; font-size: 9pt'><tr><td colspan='2' style='padding-left: 2px; border: 1px solid gray; font-weight:bold'>Historical {}th percentile values</td></tr>".format(HAZARD_THRESHOLD_PERCENTILE[1])
    for hazard_name in hazards.keys():
        html2 += '<tr><td style="padding-left: 2px; border: 1px solid gray">{0}</td><td style="padding-left: 2px; border: 1px solid gray">{1} {2}</td></tr>'.format(hazards[hazard_name]['short_displayname'], percentilebased_lookup_thresholds[hazard_name], hazards[hazard_name]['unit'])
    html2 += '</table>'
    indicator_display.set_trait('value', html2)
    
    highrisk_categories = {
        'current': [(hazcat, current_hazlevel[hazcat]) for hazcat in HAZARD_CATEGORIES if current_hazlevel[hazcat] >= LOW_MODERATE_CUTOFF],
        'future': [(hazcat, future_hazlevel[hazcat]) for hazcat in HAZARD_CATEGORIES if future_hazlevel[hazcat] >= LOW_MODERATE_CUTOFF],
        'currenthaz': [current_maxhaz[hazcat] for hazcat in HAZARD_CATEGORIES if current_hazlevel[hazcat]],
        'futurehaz': [future_maxhaz[hazcat] for hazcat in HAZARD_CATEGORIES if future_hazlevel[hazcat]]
    }
    highrisk_categories['current'].sort(key=lambda x:x[1], reverse=True)
    highrisk_categories['future'].sort(key=lambda x:x[1], reverse=True)
    highrisk_categories['current'] = [i[0] for i in highrisk_categories['current']]
    highrisk_categories['future'] = [i[0] for i in highrisk_categories['future']]

In [16]:
debug_display = widgets.HTML()
status_display = widgets.HTML()

In [17]:
time.sleep(2)
probplot.set_size_inches(5,4)

In [18]:
#update_map()
update_percentiles()
update_table()
update_probplot()
update_narrativedisplay()

In [19]:
def label(text):
    return widgets.HTML(value="<span style='font-size:24pt;font-weight:bold'>{}</span>".format(text))

In [20]:
# TAB LAYOUT
def label(text):
    return widgets.HTML(value="<span style='font-size:24pt;font-weight:bold'>{}</span>".format(text))

map_box = widgets.VBox([label('Select geography'), upload_button, Map], layout=Layout(width='600px', height='600px'))
hbar = widgets.HTML(value='<br/><div style="height:6px; background-color: #9090c0"></div>')
slider_box = widgets.VBox([threshold_intslider, threshold_floatslider], layout=Layout(align_items='flex-start'))
selector_box = widgets.VBox([label('Select timeframe'), futureyear_slider, hbar, widgets.HTML(value='<br/><div style="height:15px"></div>'), status_display], layout=Layout(width='400px', height='600px'))
indicator_box = widgets.VBox([label('Select indicator'), hazard_selector, hbar, mode_toggle, slider_box, hbar, table_display, hbar, probplot.canvas])
#threshold_box = widgets.VBox([indicator_display, widgets.HTML(value='<div style="height: 10px">'), hazardthreshold_toggle, threshold_sliderbox])
thresholdmethod_section = widgets.HBox([widgets.HTML(value="<span style='font-weight:bold; text-align:right'>Thresholds:</span>"), hazardthreshold_toggle])
thresholdvalue_container = widgets.HBox([threshold_percentiletable])
threshold_box = widgets.VBox([thresholdmethod_section, thresholdvalue_container])
hazard_box = widgets.HBox([hazard_display, widgets.HTML(value='<div style="width: 10px">'), threshold_box])
tab_area = widgets.Tab(children=[hazard_box, narrative_display, indicator_box], layout=Layout(width='600px', height='600px'))
tab_box = widgets.VBox([label('Review outputs'), tab_area])
for i in range(3):
    tab_area.set_title(i, ['Hazards', 'Hazards narrative', 'Indicator detail'][i])
dashboard = widgets.HBox([map_box, selector_box, tab_box], layout=Layout(width='1600px', height='600px'))
if mode_is_ev:
    probplot.canvas.layout.visibility = 'hidden'

In [21]:
dashboard

# Data Portal adaptation demo

**This demo implements an analytical workflow that integrates...**
* Climate-hazard probability datasets linked from Google Earth Engine
* Interactivity
* An assessment of which hazards are important at any location
* Climate-hazard magnitudes and probabilities compatible with adaptation reporting under the Global Covenant of Mayors [Common Reporting Framework](https://www.globalcovenantofmayors.org/our-initiatives/data4cities/common-global-reporting-framework/)

**What to notice**
* Important hazards are listed by hazard category, with risk levels based on probabilities of exceeding historical 75th percentiles of hazards
* Alternatively, the user can select threshold magnitudes for hazard assessment
* Current and future hazard magnitudes and probabilities are displayed in the table, the graph, and the map
* Define location of interest by placing a circle-marker, drawing a polygon, or importing a GeoJSON describing a point or polygon

**Data sources**
* Temperature and precipitation projections are calculated from [NEX-GDDP](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp) and the [ERA5](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5) daily aggregate dataset
* Landslide hazard is based on precipitation projections and hazard assessment from NASA's [LHASA](https://svs.gsfc.nasa.gov/4631) project
* Flood hazard is based on WRI's [Aqueduct Floods](https://www.wri.org/aqueduct/floods) project
* The Jupyter notebooks that make up this dashboard and the underlying analyses are in [this GitHub repo](https://github.com/wri/cities-dataportal)

**Important**
* This dashboard is still under construction
* It is very slow: responses to user inputs can take more than 30 seconds
* If you believe an error has occurred, reload the page

_Direct inquiries to [Ted Wong](https://www.wri.org/profile/ted-wong) at World Resources Institute_
